In [ ]:
import numpy as np           

import matplotlib.pyplot as plt
from scipy.interpolate import griddata

import plotly.graph_objects as go
import IPython

import gstlearn as gl
from numpy import pi, cos, sin


In [ ]:
def get_lims_colors(surfacecolor):# color limits for a slice
    return np.min(surfacecolor), np.max(surfacecolor)

def colorax(vmin, vmax):
    return dict(cmin=vmin,
                cmax=vmax)


def mapping_map_to_sphere(lon, lat,radius=6371.):
    #this function maps the points of coords (lon, lat) to points onto the  sphere of radius radius
    xs=radius*np.cos(lon-np.pi)*np.cos(lat)
    ys=radius*np.sin(lon-np.pi)*np.cos(lat)
    zs=radius*np.sin(lat)
    return xs, ys, zs


def d2r(x):
    return x * np.pi / 180.


In [ ]:
gl.variety_define(1)

mesh = gl.MeshSpherical()
mesh.reset(None,None,triswitch = "-r5",verbose=False)

X = mesh.getCoordinates(0)
Y = mesh.getCoordinates(1)



model = gl.Model.createFromParam(gl.ECov.BESSEL_K,range=50,param=1) #penser à règler range dans shiftop
S=gl.ShiftOpCs(mesh,model)
whitenoise = gl.ut_vector_simulate_gaussian(mesh.getNApices())
result = gl.VectorDouble(mesh.getNApices())
Q = gl.PrecisionOpCs(S,model.getCova(0),gl.EPowerPT.MINUSHALF)
Q.eval(whitenoise,result)


db = gl.Db.create()
db["long"]=X
db["lat"]=Y
db.setLocators(["long","lat"],gl.ELoc.X)

In [ ]:
meshes = np.array(mesh.getMeshes()).reshape([mesh.getNMeshes(),3])-1
xs,ys,zs = mapping_map_to_sphere(d2r(db["long"][:,0]),d2r(db["lat"][:,0]))

In [ ]:

cscale = [[0.0, '#313695'],
 [0.07692307692307693, '#3a67af'],
 [0.15384615384615385, '#5994c5'],
 [0.23076923076923078, '#84bbd8'],
 [0.3076923076923077, '#afdbea'],
 [0.38461538461538464, '#d8eff5'],
 [0.46153846153846156, '#d6ffe1'],
 [0.5384615384615384, '#fef4ac'],
 [0.6153846153846154, '#fed987'],
 [0.6923076923076923, '#fdb264'],
 [0.7692307692307693, '#f78249'],
 [0.8461538461538461, '#e75435'],
 [0.9230769230769231, '#cc2727'],
 [1.0, '#a50026']]


fig = go.Figure(data=[
    go.Mesh3d(
        x=xs,
        y=ys,
        z=zs,
        colorbar_title='z',
        colorscale=cscale,
        # Intensity of each vertex, which will be interpolated and color-coded
        intensity=np.array(result),
        # i, j and k give the vertices of triangles
        # here we represent the 4 triangles of the tetrahedron surface
        i=meshes[:,0],
        j=meshes[:,1],
        k=meshes[:,2],
        name='y',
        showscale=True
    )
])

fig.show()